In [31]:
# 1 - (4 points) - Via une liste chainée, une pile ou une file, créer un système de d'historique des commandes de votre bot. Ce système devra avoir comme fonctionnalités :
#     - de quoi voir la dernière commande rentrée
#     - de quoi voir toutes les commandes rentrée par un utilisateur depuis sa première connexion
#     - de quoi se déplacer dans cet historique (en avant et en arrière)
#     - de quoi vider l'historique

# 2 - (2 points) - Via une file, créer un systeme permettant de protéger l'intégrité de l'historique, pour cela il faudra trouver un moyen de limiter l'accès à l'historique pour qu'une seule personne à la fois ne puisse travailler dessus.

# 3 - (4 points) - Via un arbre binaire ou non, créer un système de discution permettant de faire un questionnaire à l'utilisateur. L'utilisateur pourra appeler une commande "help" permettant de 
# lancer la conversation et le bot tentera de définir son besoin en lui posant une série de questions prédéfinies, à la fin de la conversation une réponse sera donné au besoin. 
# les sujets que devra aborder sont libre.
#     De plus la gestion des discutions avec le bot doit avoir un certain nombre de commandes :
#     - "reset" : permettra de recommencer la discution
#     - "speak about X" : permettra de savoir si un sujet est traité par le bot ou non (exemple : speak about python dira si oui ou non le bot parle de python)

# 4 - (2 points) - Via une hashtable, stocker soit l'historique, soit l'avancement des conversations afin que les données soient liées à l'utilisateur. La clef de la table devra donc être une 
# donnée représentant l'id du compte discord d'un utilisateur et la donnée, pour l'historique par exemple, une instance de la liste chainée qui ne stockera que les commandes de cet
# utilisateur.

# 5 - (2 points) - Trouver une solution afin que lorsque que le bot s'arrête ses données stockées dans les différentes structures et collections crées précédement se soient pas perdues.
# Vous êtes libre d'utiliser ce que vous voulez pour stocker les données, un fichier texte, un fichier Json, une base de données ...

# 6 - (6 points) - Pour avoir les points restants, vous êtes libre de rajouter toutes les fonctionnalités que vous voulez à votre bot discord.

# Attention, Les listes chainées, les files, les arbres et les hashmap doivent être créés à la main et ne doivent pas venir de modules et packages python.
# Le rendu devra se faire pour le 15 Mai (date de notre dernier cours ensemble) avant minuit sous la forme d'un lien vers un repository github.
# Chaque journée de retard aura un malus de -5 points sur la note finale.

In [32]:
import nest_asyncio # async
nest_asyncio.apply()

from dotenv import load_dotenv # for .env file
import os
load_dotenv() # load .env file

# discord functions
import discord 
from discord.ext import commands

intents = discord.Intents.all() # Bot intents (permissions)
client = commands.Bot(command_prefix="/", intents = intents) # Bot prefix

In [33]:
# 1 - Via une liste chainée, une pile ou une file, créer un système de d'historique des commandes de votre bot. Ce système devra avoir comme fonctionnalités :
#     - de quoi voir la dernière commande rentrée
#     - de quoi voir toutes les commandes rentrée par un utilisateur depuis sa première connexion
#     - de quoi se déplacer dans cet historique (en avant et en arrière)
#     - de quoi vider l'historique

from Structures.stack import Stack

history = Stack('history:') # history of commands

@client.command(name="history_head")
async def print_history_head(ctx): # print the first command of the history
    await ctx.channel.send(history.peek())
    
@client.command(name="history")
async def print_history_next(ctx): # print the next command of the history
    await ctx.channel.send(history.print())
    
@client.command(name="history_clear") # NEEDS TEST
async def clear_history(ctx):
    await ctx.channel.send(history.clear())
    
@client.command(name="history_index") # NEEDS TEST
async def print_history_index(ctx):
    await ctx.channel.send("Please enter your input:")
    def check(m):
        return m.author == ctx.author and m.channel == ctx.channel
    msg = await client.wait_for("message", check=check)
    if msg.content.isdigit():
        await ctx.channel.send(history.print_index(msg.content))

In [34]:
#2 - Via une file, créer un systeme permettant de protéger l'intégrité de l'historique, pour cela il faudra trouver un moyen de limiter l'accès à l'historique pour qu'une seule personne à la fois ne puisse travailler dessus.

from Structures.queue import Queue

queue = Queue('queue:') # queue of users to access the bot

async def manage_queue():
    await client.wait_until_ready() # wait until the bot is ready
    while not client.is_closed(): # while the bot is running
        if not queue.isEmpty(): # if the queue is not empty
            user = queue.dequeue() # get the first user in the queue
            await user
        await asyncio.sleep(5)
        
async def add_to_queue(user):
    queue.enqueue(user)
    await user.send("You have been added to the queue. Please wait for your turn.")

In [35]:
# 3 - Via un arbre binaire ou non, créer un système de discution permettant de faire un questionnaire à l'utilisateur. L'utilisateur pourra appeler une commande "help" permettant de 
# lancer la conversation et le bot tentera de définir son besoin en lui posant une série de questions prédéfinies, à la fin de la conversation une réponse sera donné au besoin. 
# les sujets que devra aborder sont libre.
#     De plus la gestion des discutions avec le bot doit avoir un certain nombre de commandes :
#     - "reset" : permettra de recommencer la discution
#     - "speak about X" : permettra de savoir si un sujet est traité par le bot ou non (exemple : speak about python dira si oui ou non le bot parle de python)

from Structures.tree import Tree

def create_tree() -> Tree: 
    tree = Tree('Do you wanna play a team sport or an individualist sport ?', [], 'Which sport do you wanna play ?')
    tree.append_question('Do you wanna play a sport with a ball or without a ball ?', ['team'], 'Do you wanna play a team sport or an individualist sport ?')
    tree.append_question('Do you wanna play a sport with a racket or without a racket ?', ['individualist'], 'Do you wanna play a team sport or an individualist sport ?')
    
    tree.append_question('Do you wanna play football or basket ?', ['ball'], 'Do you wanna play a sport with a ball or without a ball ?')
    tree.append_question('Do you wanna play tennis or badminton ?', ['racket'], 'Do you wanna play a sport with a racket or without a racket ?')
    tree.append_question('Do you prefer running or swimming ?', ['without a ball'], 'Do you wanna play a sport with a ball or without a ball ?')
    tree.append_question('Do you prefer weightlifting or crossfit ', ['without a racket'], 'Do you wanna play a sport with a racket or without a racket ?')
    
    tree.append_question('Great choice football is a very famous sport for a reason', ['football'], 'Do you wanna play football or basket ?')
    tree.append_question('Great choice basket is a very famous sport for a reason', ['basket'], 'Do you wanna play football or basket ?')
    tree.append_question('Great choice tennis is a very famous sport for a reason', ['tennis'], 'Do you wanna play tennis or badminton ?')
    tree.append_question('Great choice badminton is a very famous sport for a reason', ['badminton'], 'Do you wanna play tennis or badminton ?')
    tree.append_question('Great choice running is a very famous sport for a reason', ['running'], 'Do you prefer running or swimming ?')
    tree.append_question('Great choice swimming is a very famous sport for a reason', ['swimming'], 'Do you prefer running or swimming ?')
    tree.append_question('Great choice weightlifting is a very famous sport for a reason', ['weightlifting'], 'Do you prefer weightlifting or crossfit ?')
    tree.append_question('Bad choice try again', ['crossfit'], 'Do you prefer weightlifting or crossfit ?')
    
    return tree

@client.command(name="commands")
async def help(ctx):
    await ctx.send("Hello, I'm a bot. I can help you to find a sport that you will like. Type /play to start the game. Type /reset to reset the game. Type /speak_about to know if I can speak about a sport. Type /history to see your history. Type /clear_history to clear your history. Type /queue to see the queue. Type /clear_queue to clear the queue.")
    
@client.command(name="play")
async def play(ctx): # create a new tree that is specific to the user and send the first question
    tree = create_tree()
    tree.current_node = tree.first_node
    await ctx.send(tree.get_question())
    
@client.command(name="speak_about")
async def speak_about(ctx, sport): # check if the bot can speak about a sport
    tree = create_tree()
    tree.current_node = tree.first_node
    while tree.current_node.reponses != sport:
        for N in tree.current_node.next_nodes:
            if N.reponses == sport:
                tree.current_node = N
                break
    if tree.current_node.question == sport:
        await ctx.send("Yes I can speak about " + sport)
    else:
        await ctx.send("No I can't speak about " + sport)
        
@client.command(name="reset")
async def reset(ctx): # reset the game for the user
    play(ctx)


In [36]:
#6 improvements..

@client.command(name="del")
async def delete(ctx): # delete the last 10 messages
    if queue.isEmpty():
        history.push("delete")
        await ctx.channel.purge(limit=10)
    else:
        await add_to_queue(ctx.author)
        
# @client.event
# async def on_ready():
#     print("Le bot est prêt !")

# @client.event
# async def on_typing(channel, user, when):
#      await channel.send(user.name+" is typing")

@client.event
async def on_member_join(member):
    general_channel = client.get_channel(977137496720826368)
    await general_channel.send("Bienvenue sur le serveur ! "+ member.name)

@client.event
async def on_message(message):
  if message.author == client.user:
    return   

  if message.content.startswith("hello"):
    await message.channel.send("hello")

  await client.process_commands(message)

In [37]:
client.run(os.getenv('BOT_TOKEN'))

[2023-05-10 15:27:49] [INFO    ] discord.client: logging in using static token
[2023-05-10 15:27:50] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 2330cad2e5ff0336b1d505b9a471b8bd).


play music
4 hashtable
5 store data
tests